# inforet 2024 4

## preparation

In [69]:
# !pip install rank_bm25 spacy Sense2Vec
# !wget https://github.com/explosion/sense2vec/releases/download/v1.0.0/s2v_reddit_2015_md.tar.gz
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from collections import Counter
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# from rank_bm25 import BM25Okapi
# import spacy
# from sense2vec import Sense2Vec
tqdm.pandas()

time: 1.36 ms (started: 2024-04-05 00:13:54 +00:00)


In [70]:
# this turns on the autotimer, so that every cell has a timing information below
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime
# stop using:
# %unload_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.14 ms (started: 2024-04-05 00:13:57 +00:00)


In [3]:
# 授权访问colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 2.09 s (started: 2024-04-04 23:04:33 +00:00)


In [4]:
!unzip /content/drive/MyDrive/our_msmarco.zip -d /content/our_msmarco

Archive:  /content/drive/MyDrive/our_msmarco.zip
replace /content/our_msmarco/our.msmarco.docs.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/our_msmarco/our.msmarco.docs.tsv  y
y
y
y

replace /content/our_msmarco/our.msmarco.queries.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: /content/our_msmarco/our.msmarco.queries.tsv  
replace /content/our_msmarco/our.msmarco.gold.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: /content/our_msmarco/our.msmarco.gold.tsv  
time: 1min 24s (started: 2024-04-04 23:04:35 +00:00)


## simplifying the data and writing it into a file

In [71]:
import os
os.chdir('/content/our_msmarco/')

time: 673 µs (started: 2024-04-05 00:14:06 +00:00)


In [72]:
origdocs = pd.read_csv('our.msmarco.docs.tsv',sep='\t',usecols=[1,2,3])
origdocs['title'].fillna('-', inplace=True)
origdocs['body'].fillna('-', inplace=True)
origdocs

,docid,title,body
0,D2981241,What do you call a group of lions?,Lions Vocabulary of the English Language Word ...
1,D687756,.,"The A Priori Argument ( also, Rationalization;..."
2,D913099,Everything You Need To Learn How To Cook Veget...,Home > How To Cook Vegetables Everything You N...
3,D328017,"What is the difference between latitude, longi...",Longitude Latitude Geographic Coordinate Syste...
4,D1636347,When was the pulley invented?,Answers.com ® Wiki Answers ® Categories Techno...
...,...,...,...
92560,D3379210,Top 39 Doctor insights on: Can An Iud Cause Ha...,Top 39 Doctor insights on: Can An Iud Cause Ha...
92561,D3068739,How to get back your DirecTV cancellation fees,How to get back your Direc TV cancellation fee...
92562,D1590402,Certification FAQs,Fingerprinting 1. Where can I get fingerprinte...
92563,D2175490,Greenhouse gas emissions by Canadian economic ...,"Access PDF (682 KB)In 2015, Canada's total gre..."


time: 25.6 s (started: 2024-04-05 00:14:08 +00:00)


In [7]:
ourdocs = pd.DataFrame(columns = ['docid', 'text'])
ourdocs['docid']=origdocs.docid
ourdocs['text']=origdocs.title+' '+origdocs.body
ourdocs

,docid,text
0,D2981241,What do you call a group of lions? Lions Vocab...
1,D687756,". The A Priori Argument ( also, Rationalizatio..."
2,D913099,Everything You Need To Learn How To Cook Veget...
3,D328017,"What is the difference between latitude, longi..."
4,D1636347,When was the pulley invented? Answers.com ® Wi...
...,...,...
92560,D3379210,Top 39 Doctor insights on: Can An Iud Cause Ha...
92561,D3068739,How to get back your DirecTV cancellation fees...
92562,D1590402,Certification FAQs Fingerprinting 1. Where can...
92563,D2175490,Greenhouse gas emissions by Canadian economic ...


time: 11.7 s (started: 2024-04-04 23:06:25 +00:00)


In [8]:
del origdocs # saving memory

time: 310 µs (started: 2024-04-04 23:06:37 +00:00)


In [9]:
# ~ 30 seconds
ourdocs.to_csv('our.text.msmarco.docs.tsv',sep='\t', columns=['docid','text'])

time: 28.5 s (started: 2024-04-04 23:06:37 +00:00)


## vectorize

In [10]:
# long: takes > 3min
vectorizer = TfidfVectorizer(sublinear_tf=True, strip_accents='unicode')
X = vectorizer.fit_transform(ourdocs.text)
print('vectorization is done', X.shape)

vectorization is done (92565, 2067446)
time: 5min 3s (started: 2024-04-04 23:07:06 +00:00)


In [11]:
queries = pd.read_csv('our.msmarco.queries.tsv',sep='\t',usecols=[1,2])
training_queries=queries.iloc[:500]
testing_queries=queries.iloc[500:]
training_queries

,qid,query
0,687888,what is a jpe
1,480210,price for asphalt driveway
2,591004,what causes pressure skin bruising
3,260536,how long drive from flagstaff to grand canyon
4,39422,average number of bowel movements per day for ...
...,...,...
495,133970,definition of dietary fiber
496,79788,can you start up a video record?
497,791583,what is rheumatoid spondylosis
498,732078,what is coleman fuel made out of


time: 16.4 ms (started: 2024-04-04 23:12:09 +00:00)


In [12]:
train_qvs = vectorizer.transform(training_queries['query'])
test_qvs = vectorizer.transform(testing_queries['query'])
#
train_qvs.shape

(500, 2067446)

time: 44.9 ms (started: 2024-04-04 23:12:09 +00:00)


### 🚧 todo:
what happens for words that are in a query and not in the documents?

-- Unknown words can be simply ignored, as the remaining known words may provide enough context to still return relevant results or make accurate predictions. Or use synonym mapping to map unknown words to known words.

What use transform and not fit_transform?

-- Use transform when applying the same transformation parameters learned from the training data (fit) to new data. It ensures that the preprocessing of training data and any new data is consistent, thus avoiding data leaks and ensuring that the model performs as expected on unseen data.

In [13]:
train_qvs[0].shape

(1, 2067446)

time: 2.48 ms (started: 2024-04-04 23:12:10 +00:00)


In [14]:
gold = pd.read_csv('our.msmarco.gold.tsv',sep='\t',usecols=[1,3,4,5])
gold

,qid,docid,rank,score
0,310290,D579750,1,-5.11498
1,310290,D579754,2,-5.57703
2,310290,D2380815,3,-5.84852
3,310290,D822566,4,-5.95002
4,310290,D2249695,5,-6.08326
...,...,...,...,...
99995,257942,D253854,96,-6.32693
99996,257942,D3056621,97,-6.32837
99997,257942,D1323491,98,-6.32871
99998,257942,D2722485,99,-6.33100


time: 169 ms (started: 2024-04-04 23:12:10 +00:00)


# scores

In [15]:
qid = 257942
# understanding the score column:
gold[(gold.qid==qid)]

,qid,docid,rank,score
99900,257942,D1451841,1,-5.66844
99901,257942,D1361861,2,-5.75280
99902,257942,D2627169,3,-5.76717
99903,257942,D3074165,4,-5.86465
99904,257942,D1361864,5,-5.95433
...,...,...,...,...
99995,257942,D253854,96,-6.32693
99996,257942,D3056621,97,-6.32837
99997,257942,D1323491,98,-6.32871
99998,257942,D2722485,99,-6.33100


time: 16.7 ms (started: 2024-04-04 23:12:10 +00:00)


In [16]:
# getting the top 10 documents for each query
qgold = gold[(gold.qid==qid) & (gold['rank']<=10)].docid
qgold

99900    D1451841
99901    D1361861
99902    D2627169
99903    D3074165
99904    D1361864
99905    D2531901
99906    D1624776
99907    D2926659
99908    D1805809
99909     D885919
Name: docid, dtype: object

time: 5.95 ms (started: 2024-04-04 23:12:10 +00:00)


In [17]:
# getting the relevance score for the top document for the query
gold[(gold.qid==qid) & (gold.docid==qgold.iloc[0])].score

99900   -5.66844
Name: score, dtype: float64

time: 15.5 ms (started: 2024-04-04 23:12:10 +00:00)


In [18]:
# getting the average relevance score for all top results of test queries
def get_avg_score(testing_queries, top=10):
    return gold[(gold.qid.isin(testing_queries.qid)) & (gold['rank']<=top)].score.mean()

for top in [1, 2, 3, 4, 5, 10, 100]:
    print(top, get_avg_score(testing_queries, top))



1 -5.197412539999999
2 -5.27601475
3 -5.33337584
4 -5.378957215000001
5 -5.416540648
10 -5.543673964
100 -6.049410218599999
time: 21.1 ms (started: 2024-04-04 23:12:10 +00:00)


# case study

In [19]:
# can we improve the relevance score by using a dense embedding?
training_queries[training_queries.qid==251898]



,qid,query
111,251898,how long does getting a doctorate take


time: 9.37 ms (started: 2024-04-04 23:12:10 +00:00)


In [20]:
gold[gold.qid==251898]

,qid,docid,rank,score
36200,251898,D2865964,1,-4.74293
36201,251898,D3557816,2,-4.90695
36202,251898,D2723985,3,-4.95911
36203,251898,D1951655,4,-4.97272
36204,251898,D1709749,5,-5.02176
...,...,...,...,...
36295,251898,D2531901,96,-5.56896
36296,251898,D2956542,97,-5.57138
36297,251898,D301873,98,-5.57262
36298,251898,D2952336,99,-5.57504


time: 15.5 ms (started: 2024-04-04 23:12:10 +00:00)


time: 20.5 ms (started: 2024-04-04 23:12:10 +00:00)


In [21]:
# to remove for todo
qid = 729561
gold[gold.qid==qid].docid
# show lines of ourdocs where docid is in gold[gold.qid==qid].docid
ourdocs[ourdocs.docid.isin(gold[(gold.qid == qid) & (gold['rank'] <= 20)].docid)]
# gold[(gold.qid == qid) & (gold['rank'] <= 10)]
df = ourdocs[ourdocs.docid.isin(gold[(gold.qid == qid) & (gold['rank'] <= 20)].docid)].copy()
df['counter'] = range(1, len(df) + 1)
df

,docid,text,counter
737,D2696063,What tv channel is syfy on if you have dish ne...,1
2277,D2516893,Chapter 12-Nervous System 129 terms rotilla10C...,2
2322,D588822,What channel is NBC on XFINITY? Comcast Xfinit...,3
2955,D1422234,What channel is the yes network on comcast in ...,4
10801,D831199,Chapter 7: Nerve Cells Chapter 7: Nerve Cells8...,5
10953,D214463,What channel is wgn on Verizon fios? C.skater ...,6
15313,D1642934,Types of Receptors 38 terms brittney_harvell T...,7
15749,D1137136,What channel is tnt on dish network? Answers.c...,8
22362,D893699,What channel is wgn on Verizon fios? C.skater ...,9
22632,D1203964,What channel is CBS for DISH Network? Answers....,10


time: 43.1 ms (started: 2024-04-04 23:12:10 +00:00)


### 🚧 todo:
- complete the xxx in the lines
- comment each line

In [22]:

def show_doc(qid, rank=0, show_gold=False, k=10, similarities=None, docs=ourdocs, gold=gold):
    """
    qid: query id
    rank: rank of the document to show. if rank=0, show the top k documents
    show_gold: if True, show the gold documents. if False, show the predicted documents
    k: number of documents to show
    docs: documents DataFrame
    gold: gold DataFrame
    """
    print(f"query: {training_queries[training_queries.qid==qid]['query'].values[0]}")
    id = training_queries[training_queries.qid==qid].index[0] # get the index of the query in the training_queries
    xqv = X*train_qvs[id].T # cosine similarity between query and all documents
    predki = np.argpartition(xqv.A.flat, -k)[-k:] # top k
    if show_gold:
        if rank == 0:
            print(f"top {k} gold documents for query {qid}:")
            df = ourdocs[ourdocs.docid.isin(gold[(gold.qid == qid) & (gold['rank'] <= k)].docid)].copy() # Create a copy to avoid modifying the original DataFrame
            pred10i = np.argpartition(xqv.A.flat, -10)[-10:] # top 10
            pred100i = np.argpartition(xqv.A.flat, -100)[-100:] # top 100
            df['in_top_10_pred'] = df['docid'].isin(docs.loc[pred10i].docid)
            df['in_top_100_pred'] = df['docid'].isin(docs.loc[pred100i].docid)
            display(df)
        else:
            print(f"gold document {rank} for query {qid}:")
            display(ourdocs[ourdocs.docid==gold[(gold.qid==qid) & (gold['rank']==rank)].docid.iloc[0]].text.values[0][:999])
    else: # show predictions
        if rank == 0:
            print(f"top {k} predicted documents for query {qid}:")

            qgold10 = gold[(gold.qid == qid) & (gold['rank'] <= 10)].docid
            qgold100 = gold[(gold.qid == qid) & (gold['rank'] <= 100)].docid

            df = docs.loc[predki].copy()  # Create a copy to avoid modifying the original DataFrame
            df['in_qgold10'] = df['docid'].isin(qgold10)
            df['in_qgold100'] = df['docid'].isin(qgold100)
            if similarities is not None:
                df['similarity'] = similarities
                df = df.sort_values(by='similarity', ascending=False)
            display(df)
        else:
            print(f"predicted document {rank} for query {qid}:")
            display(ourdocs[ourdocs.docid==docs.loc[predki].docid.iloc[rank]].text.values[0][:999])


show_doc(729561,rank=7)
show_doc(729561)
# show_doc(729561,show_gold=True, k=20)

query: what is channeling
predicted document 7 for query 729561:


". Arc Resistant Switchgear ANSI/IEEE C37.20.7 defines switchgear arc resistance in two basic categories: ANSI type 1 Arc resistance from the front of gear only ANSI type 2 Arc resistance provided from the front, sides and rear A suffix may be added to either of these two types to further define the type of protection provided: A: Basic design B: Arc resistance is maintained even while opening designated low voltage compartments C: Arc resistance is maintained even when opening designated adjacent compartments D: Special designation that supplements the Type 1 designation, but identifies additional arc resistance in certain structures Eaton offers arc resistant medium voltage (MV),and low voltage (LV) switchgear and arc resistant MV motor control. Eaton also offers a unique “arc preventative” LV motor control center (Flash Gard) that protects the operator by reducing the likelihood that an arc would occur. Arc Resistant MV Switchgear Eaton's VCP-W arc-resistant vacuum switchgear (5 k V

query: what is channeling
top 10 predicted documents for query 729561:


,docid,text,in_qgold10,in_qgold100
39754,D858701,Polar Bears Polar Bears Spending the dark wint...,False,False
25371,D461511,Earrings dream meaning Earrings dream meaning ...,False,False
15563,D2167062,What to do with lemon curd. Are you channeling...,False,False
64517,D3400862,FAQ FAQHow do I select the right replacement t...,False,False
49665,D1681032,Behind the gates at Ramtha's school By Lisa Pe...,False,False
86431,D567760,"Roosevelt Roosevelt [ roh -z uh -velt, -v uh l...",False,False
65542,D873198,"The Magician The Magician February 27, 2014 by...",False,False
56830,D1636027,. Arc Resistant Switchgear ANSI/IEEE C37.20.7 ...,False,False
67662,D685247,What is Channeling? What is Channeling? By SHE...,False,True
34985,D1690699,- Are you channeling your best self with this ...,False,False


time: 775 ms (started: 2024-04-04 23:12:10 +00:00)


#### 2nd case:

In [23]:
show_doc(251898)


query: how long does getting a doctorate take
top 10 predicted documents for query 251898:


,docid,text,in_qgold10,in_qgold100
50610,D2749717,How long does it take to become a Physical The...,False,True
56982,D2865964,How long does it take to get a post-doctoral d...,True,True
83787,D1951655,How long does it take to get a Doctorate of Nu...,True,True
67623,D2723985,How long would it take to get a doctorate's de...,True,True
66313,D2956542,How long does it take to finish a doctorate? E...,False,True
86925,D2703712,How Long Does It Take To Become A Psychologist...,False,True
52957,D1402632,How to get doctorate degree How to get doctora...,False,True
42225,D2782441,How many years does a doctorate degree require...,False,True
65033,D1736725,How long does it take to get a doctoral degree...,True,True
91275,D3090762,How Long Does It Take to Get a Doctorate in En...,False,True


time: 382 ms (started: 2024-04-04 23:12:11 +00:00)


#### TODO: 3rd case:
- find yourself another case where the retrieved top 10 contains interesting documents but the order is not good and the top 1 is not relevant


In [53]:
show_doc(310290)

query: how much does a mclaren cost
top 10 predicted documents for query 310290:


,docid,text,in_qgold10,in_qgold100
16782,D3549914,How much does fish cost? Colee Randens 6 Contr...,False,False
14954,D882643,Freehold Cartage Inc Freehold Cartage Inc@Free...,False,False
74684,D579754,How Much Does a Mclaren F1 Cost? How Much Does...,True,True
68677,D822566,"How fast can mclaren f1 go? mike P 3,945 Contr...",True,True
26578,D2380815,"F1 Info Guide (Pt. 1): Money, Money, MoneyâThe...",True,True
58928,D822565,What car company makes the mclaren cars? Cars ...,False,True
42833,D1879939,Take a closer look at the McLaren F1 â the mos...,False,True
3086,D1640175,Does Chrysler still own Mercedes Benz? Answers...,True,True
17103,D579750,How much does a Mercedes-Benz SLR Mclaren Stir...,True,True
52455,D1517648,Who makes Mclarens? BMW or Mercedes? Cars & Tr...,False,True


time: 548 ms (started: 2024-04-04 23:41:24 +00:00)


I modified the show_doc function to print df

In [61]:

def show_doc1(qid, rank=0, show_gold=False, k=10, similarities=None, docs=ourdocs, gold=gold):
    """
    qid: query id
    rank: rank of the document to show. if rank=0, show the top k documents
    show_gold: if True, show the gold documents. if False, show the predicted documents
    k: number of documents to show
    docs: documents DataFrame
    gold: gold DataFrame
    """
    # print(f"query: {training_queries[training_queries.qid==qid]['query'].values[0]}")
    id = training_queries[training_queries.qid==qid].index[0] # get the index of the query in the training_queries
    xqv = X*train_qvs[id].T # cosine similarity between query and all documents
    predki = np.argpartition(xqv.A.flat, -k)[-k:] # top k
    if show_gold:
        if rank == 0:
            print(f"top {k} gold documents for query {qid}:")
            df = ourdocs[ourdocs.docid.isin(gold[(gold.qid == qid) & (gold['rank'] <= k)].docid)].copy() # Create a copy to avoid modifying the original DataFrame
            pred10i = np.argpartition(xqv.A.flat, -10)[-10:] # top 10
            pred100i = np.argpartition(xqv.A.flat, -100)[-100:] # top 100
            df['in_top_10_pred'] = df['docid'].isin(docs.loc[pred10i].docid)
            df['in_top_100_pred'] = df['docid'].isin(docs.loc[pred100i].docid)
            return df
        else:
            # print(f"gold document {rank} for query {qid}:")
            # display(ourdocs[ourdocs.docid==gold[(gold.qid==qid) & (gold['rank']==rank)].docid.iloc[0]].text.values[0][:999])
            return ourdocs[ourdocs.docid==gold[(gold.qid==qid) & (gold['rank']==rank)].docid.iloc[0]].text.values[0][:999]
    else: # show predictions
        if rank == 0:
            # print(f"top {k} predicted documents for query {qid}:")

            qgold10 = gold[(gold.qid == qid) & (gold['rank'] <= 10)].docid
            qgold100 = gold[(gold.qid == qid) & (gold['rank'] <= 100)].docid

            df = docs.loc[predki].copy()  # Create a copy to avoid modifying the original DataFrame
            df['in_qgold10'] = df['docid'].isin(qgold10)
            df['in_qgold100'] = df['docid'].isin(qgold100)
            if similarities is not None:
                df['similarity'] = similarities
                df = df.sort_values(by='similarity', ascending=False)
            # display(df)
            return df
        else:
            # print(f"predicted document {rank} for query {qid}:")
            # display(ourdocs[ourdocs.docid==docs.loc[predki].docid.iloc[rank]].text.values[0][:999])
            return ourdocs[ourdocs.docid==docs.loc[predki].docid.iloc[rank]].text.values[0][:999]



time: 1.39 ms (started: 2024-04-05 00:07:24 +00:00)


In [74]:
def find_case_with_specific_conditions(queries):
    for qid in queries['qid']:
        filtered_df = show_doc1(qid)
        # if filtered_df['in_qgold10'].all() == False and filtered_df['in_qgold100'].any() == True:
        #   print(qid)
        #   return qid
        if not filtered_df.empty and 'in_qgold10' in filtered_df.columns and 'in_qgold100' in filtered_df.columns:
            # Check if the first 'in_qgold10' entry is False
            first_in_qgold10_is_false = filtered_df.iloc[0]['in_qgold10'] == False
            at_least_one_in_qgold10_is_true = filtered_df['in_qgold10'].any()
            at_least_one_in_qgold100_is_true = filtered_df['in_qgold100'].any()

            # If both conditions are met, print and return the qid
            if first_in_qgold10_is_false and at_least_one_in_qgold100_is_true and at_least_one_in_qgold10_is_true:
                print(qid)
                return qid





result = find_case_with_specific_conditions(queries)



480210
time: 826 ms (started: 2024-04-05 00:15:18 +00:00)


In [75]:
show_doc(480210)

query: price for asphalt driveway
top 10 predicted documents for query 480210:


,docid,text,in_qgold10,in_qgold100
26057,D513755,Compare Asphalt vs Concrete Driveway Costs Com...,False,True
27770,D1706519,How much does a new asphalt driveway cost? Hom...,False,True
58431,D258785,How Thick Should An Asphalt Driveway Be? << Ba...,False,True
78500,D2950267,How Thick Should An Asphalt Driveway Be? << Ba...,False,True
66592,D1774629,What Is the Average Cost of an Asphalt Drivewa...,False,True
61338,D2704669,When to Seal a New Asphalt Driveway When to Se...,False,True
45695,D2476000,How to Calculate Cost of Asphalting a Driveway...,False,True
45024,D525693,How to Calculate Cost of Asphalting a Driveway...,False,True
90020,D54102,Asphalt Driveway Cost Guide for Repairs and Re...,True,True
75923,D163572,Can You Asphalt Pave Driveway in Winter Can Yo...,False,True


time: 380 ms (started: 2024-04-05 00:15:21 +00:00)


# dense embeddings

#### sample:
from https://www.mixedbread.ai/blog/mxbai-embed-large-v1

In [25]:
!pip install -U mixedbread-ai sentence-transformers


time: 5.43 s (started: 2024-04-04 23:12:16 +00:00)


In [78]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

# 1. load model
model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

# For retrieval you need to pass this prompt.
query = 'Represent this sentence for searching relevant passages: A man is eating a piece of bread'

sampledocs = [
    query,
    "A man is eating food.",
    "A man is eating pasta.",
    "The girl is carrying a baby.",
    "A man is riding a horse.",
]

time: 1.92 s (started: 2024-04-05 00:16:47 +00:00)


In [79]:
# 2. Encode
embeddings = model.encode(sampledocs)

similarities = cos_sim(embeddings[0], embeddings[1:])
print('similarities:', similarities)

similarities: tensor([[0.7920, 0.6369, 0.1651, 0.3621]])
time: 33.1 ms (started: 2024-04-05 00:16:52 +00:00)


### now to our case with Ms-Marco

In [80]:
k=10
id = training_queries[training_queries.qid==qid].index[0] # get the index of the query in the training_queries
xqv = X*train_qvs[id].T # cosine similarity between query and all documents
predki = np.argpartition(xqv.A.flat, -k)[-k:] # top k
ourdocs.loc[predki].text


26057    Compare Asphalt vs Concrete Driveway Costs Com...
27770    How much does a new asphalt driveway cost? Hom...
58431    How Thick Should An Asphalt Driveway Be? << Ba...
78500    How Thick Should An Asphalt Driveway Be? << Ba...
66592    What Is the Average Cost of an Asphalt Drivewa...
61338    When to Seal a New Asphalt Driveway When to Se...
45695    How to Calculate Cost of Asphalting a Driveway...
45024    How to Calculate Cost of Asphalting a Driveway...
90020    Asphalt Driveway Cost Guide for Repairs and Re...
75923    Can You Asphalt Pave Driveway in Winter Can Yo...
Name: text, dtype: object

time: 438 ms (started: 2024-04-05 00:16:57 +00:00)


In [81]:
query = f"Represent this sentence for searching relevant passages: {training_queries[training_queries.qid==qid]['query'].values[0]}"
print(query)


Represent this sentence for searching relevant passages: price for asphalt driveway
time: 1.64 ms (started: 2024-04-05 00:17:01 +00:00)


In [82]:
sampledocs = [query] + ourdocs.loc[predki].text.to_list()
embeddings = model.encode(sampledocs) # this takes > a second per document although the context size is only 512 tokens


time: 363 ms (started: 2024-04-05 00:17:04 +00:00)


In [83]:
similarities = cos_sim(embeddings[0], embeddings[1:])
print('similarities:', similarities[0])

similarities: tensor([0.8511, 0.8766, 0.6944, 0.6944, 0.8480, 0.6887, 0.7848, 0.7848, 0.8277,
        0.7007])
time: 4.14 ms (started: 2024-04-05 00:17:11 +00:00)


In [84]:
df = ourdocs.loc[predki].copy()  # Create a copy to avoid modifying the original DataFrame
df['similarity'] = similarities[0]
df = df.sort_values(by='similarity', ascending=False)
df

,docid,text,similarity
27770,D1706519,How much does a new asphalt driveway cost? Hom...,0.876646
26057,D513755,Compare Asphalt vs Concrete Driveway Costs Com...,0.851140
66592,D1774629,What Is the Average Cost of an Asphalt Drivewa...,0.848013
90020,D54102,Asphalt Driveway Cost Guide for Repairs and Re...,0.827666
45695,D2476000,How to Calculate Cost of Asphalting a Driveway...,0.784790
45024,D525693,How to Calculate Cost of Asphalting a Driveway...,0.784790
75923,D163572,Can You Asphalt Pave Driveway in Winter Can Yo...,0.700744
58431,D258785,How Thick Should An Asphalt Driveway Be? << Ba...,0.694360
78500,D2950267,How Thick Should An Asphalt Driveway Be? << Ba...,0.694360
61338,D2704669,When to Seal a New Asphalt Driveway When to Se...,0.688740


time: 21.9 ms (started: 2024-04-05 00:17:35 +00:00)


In [85]:
show_doc(729561, similarities=similarities[0])


query: what is channeling
top 10 predicted documents for query 729561:


,docid,text,in_qgold10,in_qgold100,similarity
25371,D461511,Earrings dream meaning Earrings dream meaning ...,False,False,0.876646
39754,D858701,Polar Bears Polar Bears Spending the dark wint...,False,False,0.851140
49665,D1681032,Behind the gates at Ramtha's school By Lisa Pe...,False,False,0.848013
67662,D685247,What is Channeling? What is Channeling? By SHE...,False,True,0.827666
65542,D873198,"The Magician The Magician February 27, 2014 by...",False,False,0.784790
56830,D1636027,. Arc Resistant Switchgear ANSI/IEEE C37.20.7 ...,False,False,0.784790
34985,D1690699,- Are you channeling your best self with this ...,False,False,0.700744
15563,D2167062,What to do with lemon curd. Are you channeling...,False,False,0.694360
64517,D3400862,FAQ FAQHow do I select the right replacement t...,False,False,0.694360
86431,D567760,"Roosevelt Roosevelt [ roh -z uh -velt, -v uh l...",False,False,0.688740


time: 463 ms (started: 2024-04-05 00:17:39 +00:00)


#### 2nd case

In [86]:
qid = 251898
show_doc(qid)


query: how long does getting a doctorate take
top 10 predicted documents for query 251898:


,docid,text,in_qgold10,in_qgold100
50610,D2749717,How long does it take to become a Physical The...,False,True
56982,D2865964,How long does it take to get a post-doctoral d...,True,True
83787,D1951655,How long does it take to get a Doctorate of Nu...,True,True
67623,D2723985,How long would it take to get a doctorate's de...,True,True
66313,D2956542,How long does it take to finish a doctorate? E...,False,True
86925,D2703712,How Long Does It Take To Become A Psychologist...,False,True
52957,D1402632,How to get doctorate degree How to get doctora...,False,True
42225,D2782441,How many years does a doctorate degree require...,False,True
65033,D1736725,How long does it take to get a doctoral degree...,True,True
91275,D3090762,How Long Does It Take to Get a Doctorate in En...,False,True


time: 461 ms (started: 2024-04-05 00:17:43 +00:00)


In [87]:
k=10
id = training_queries[training_queries.qid==qid].index[0] # get the index of the query in the training_queries
xqv = X*train_qvs[id].T # cosine similarity between query and all documents
predki = np.argpartition(xqv.A.flat, -k)[-k:] # top k
ourdocs.loc[predki].text
query = f"Represent this sentence for searching relevant passages: {training_queries[training_queries.qid==qid]['query'].values[0]}"
sampledocs = [query] + ourdocs.loc[predki].text.to_list()
embeddings = model.encode(sampledocs)
similarities = cos_sim(embeddings[0], embeddings[1:])
show_doc(qid, similarities=similarities[0])


query: how long does getting a doctorate take
top 10 predicted documents for query 251898:


,docid,text,in_qgold10,in_qgold100,similarity
66313,D2956542,How long does it take to finish a doctorate? E...,False,True,0.843658
91275,D3090762,How Long Does It Take to Get a Doctorate in En...,False,True,0.840919
65033,D1736725,How long does it take to get a doctoral degree...,True,True,0.837360
56982,D2865964,How long does it take to get a post-doctoral d...,True,True,0.813156
42225,D2782441,How many years does a doctorate degree require...,False,True,0.804683
67623,D2723985,How long would it take to get a doctorate's de...,True,True,0.773761
83787,D1951655,How long does it take to get a Doctorate of Nu...,True,True,0.756096
52957,D1402632,How to get doctorate degree How to get doctora...,False,True,0.741600
86925,D2703712,How Long Does It Take To Become A Psychologist...,False,True,0.721173
50610,D2749717,How long does it take to become a Physical The...,False,True,0.667144


time: 1.26 s (started: 2024-04-05 00:17:49 +00:00)


### 🚧 todo:
- do your 3rd case and comment on the results.

In [88]:
qid = 480210
show_doc(qid)

query: price for asphalt driveway
top 10 predicted documents for query 480210:


,docid,text,in_qgold10,in_qgold100
26057,D513755,Compare Asphalt vs Concrete Driveway Costs Com...,False,True
27770,D1706519,How much does a new asphalt driveway cost? Hom...,False,True
58431,D258785,How Thick Should An Asphalt Driveway Be? << Ba...,False,True
78500,D2950267,How Thick Should An Asphalt Driveway Be? << Ba...,False,True
66592,D1774629,What Is the Average Cost of an Asphalt Drivewa...,False,True
61338,D2704669,When to Seal a New Asphalt Driveway When to Se...,False,True
45695,D2476000,How to Calculate Cost of Asphalting a Driveway...,False,True
45024,D525693,How to Calculate Cost of Asphalting a Driveway...,False,True
90020,D54102,Asphalt Driveway Cost Guide for Repairs and Re...,True,True
75923,D163572,Can You Asphalt Pave Driveway in Winter Can Yo...,False,True


time: 474 ms (started: 2024-04-05 00:17:54 +00:00)


### 🚧 todo:
- do an evaluation on our 3 cases to see whether the relevance score improves.

In [90]:
k=10
id = training_queries[training_queries.qid==qid].index[0] # get the index of the query in the training_queries
xqv = X*train_qvs[id].T # cosine similarity between query and all documents
predki = np.argpartition(xqv.A.flat, -k)[-k:] # top k
ourdocs.loc[predki].text
query = f"Represent this sentence for searching relevant passages: {training_queries[training_queries.qid==qid]['query'].values[0]}"
sampledocs = [query] + ourdocs.loc[predki].text.to_list()
embeddings = model.encode(sampledocs)
similarities = cos_sim(embeddings[0], embeddings[1:])
show_doc(qid, similarities=similarities[0])

query: price for asphalt driveway
top 10 predicted documents for query 480210:


,docid,text,in_qgold10,in_qgold100,similarity
27770,D1706519,How much does a new asphalt driveway cost? Hom...,False,True,0.876646
26057,D513755,Compare Asphalt vs Concrete Driveway Costs Com...,False,True,0.851140
66592,D1774629,What Is the Average Cost of an Asphalt Drivewa...,False,True,0.848013
90020,D54102,Asphalt Driveway Cost Guide for Repairs and Re...,True,True,0.827666
45695,D2476000,How to Calculate Cost of Asphalting a Driveway...,False,True,0.784790
45024,D525693,How to Calculate Cost of Asphalting a Driveway...,False,True,0.784790
75923,D163572,Can You Asphalt Pave Driveway in Winter Can Yo...,False,True,0.700744
58431,D258785,How Thick Should An Asphalt Driveway Be? << Ba...,False,True,0.694360
78500,D2950267,How Thick Should An Asphalt Driveway Be? << Ba...,False,True,0.694360
61338,D2704669,When to Seal a New Asphalt Driveway When to Se...,False,True,0.688740


time: 1.5 s (started: 2024-04-05 00:18:45 +00:00)


### check out the tokenization

In [91]:
from transformers import AutoModel, AutoTokenizer
model_id = 'mixedbread-ai/mxbai-embed-large-v1'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id).cuda()


time: 1.08 s (started: 2024-04-05 00:19:31 +00:00)


In [92]:
long_words = [
    "pneumonoultramicroscopicsilicovolcanoconiosis",  # A lung disease caused by inhaling silica dust
    "pseudopseudohypoparathyroidism",             # A rare genetic disorder
    "hippopotomonstrosesquippedaliophobia",         # Fear of long words (interestingly enough)
    "anticonstitutionallyestablishmentarianism",  # Opposition to establishing a constitution
    "supercalifragilisticexpialidocious",          # From Mary Poppins (not actually that long!)
    "floccinaucinihilipilification",              # The act of estimating something as worthless
    "electroencephalogram",                         # Recording electrical activity of the brain
    "rhinencephalospheropneumonectomy",            # Surgical removal of part of the brain and lung
    "incomprehensibilities",                       # The state of being impossible to understand
    "honorificabilitudinitatibus",                 # (Shakespearean) A long word for "worthiness"
]

' '.join(tokenizer.tokenize('Hello wondrous information retrieval world: Oh, an unseenword!'+' '.join(long_words)))

'hello won ##dro ##us information retrieval world : oh , an unseen ##word ! p ##ne ##um ##ono ##ult ##ram ##ic ##ros ##copic ##sil ##ico ##vo ##lc ##ano ##con ##ios ##is pseudo ##pse ##ud ##oh ##yp ##opa ##rath ##yr ##oid ##ism hip ##pop ##oto ##mons ##tro ##ses ##qui ##pped ##ali ##op ##ho ##bia anti ##con ##sti ##tu ##tion ##ally ##est ##ab ##lish ##ment ##arian ##ism super ##cal ##if ##rag ##ilis ##tic ##ex ##pia ##lid ##oc ##ious fl ##oc ##cina ##uc ##ini ##hil ##ip ##ili ##fication electro ##ence ##pha ##logram rhine ##nce ##pha ##los ##pher ##op ##ne ##um ##one ##ct ##omy inc ##omp ##re ##hen ##si ##bilities honor ##ific ##abi ##lit ##udi ##ni ##tat ##ib ##us'

time: 4.31 ms (started: 2024-04-05 00:19:33 +00:00)


### 🚧 todo:
- write a function that gives a list of tokens for a given words that are split up by the tokenizer

In [93]:
def tokenize_words(word_list):
    # Tokenize each word and store tokens in a list
    tokenized_words = [' '.join(tokenizer.tokenize(word)) for word in word_list]
    return tokenized_words

time: 569 µs (started: 2024-04-05 00:20:52 +00:00)


In [94]:
# Tokenize list of long words
tokenized_long_words = tokenize_words(long_words)

# printed
for word, tokens in zip(long_words, tokenized_long_words):
    print(f"Word: {word}\nTokens: {tokens}\n")


Word: pneumonoultramicroscopicsilicovolcanoconiosis
Tokens: p ##ne ##um ##ono ##ult ##ram ##ic ##ros ##copic ##sil ##ico ##vo ##lc ##ano ##con ##ios ##is

Word: pseudopseudohypoparathyroidism
Tokens: pseudo ##pse ##ud ##oh ##yp ##opa ##rath ##yr ##oid ##ism

Word: hippopotomonstrosesquippedaliophobia
Tokens: hip ##pop ##oto ##mons ##tro ##ses ##qui ##pped ##ali ##op ##ho ##bia

Word: anticonstitutionallyestablishmentarianism
Tokens: anti ##con ##sti ##tu ##tion ##ally ##est ##ab ##lish ##ment ##arian ##ism

Word: supercalifragilisticexpialidocious
Tokens: super ##cal ##if ##rag ##ilis ##tic ##ex ##pia ##lid ##oc ##ious

Word: floccinaucinihilipilification
Tokens: fl ##oc ##cina ##uc ##ini ##hil ##ip ##ili ##fication

Word: electroencephalogram
Tokens: electro ##ence ##pha ##logram

Word: rhinencephalospheropneumonectomy
Tokens: rhine ##nce ##pha ##los ##pher ##op ##ne ##um ##one ##ct ##omy

Word: incomprehensibilities
Tokens: inc ##omp ##re ##hen ##si ##bilities

Word: honorificabilitu

### other things that we could look into (if we had the time and the computational resources):
- easy:
  - use the mixedbread reranking feature https://www.mixedbread.ai/blog/mxbai-rerank-v1 (see below)


- hard
  - complete text embeddings beyond 512 tokens
    - and then saving and sharing precomputed embeddings
  - using only this embedding on all documents and compare it with tf-idf/bm25


In [97]:
pip install -U mixedbread-ai

time: 5.43 s (started: 2024-04-05 00:21:57 +00:00)


time: 34.9 ms (started: 2024-04-04 23:12:32 +00:00)


time: 40.8 ms (started: 2024-04-04 23:12:32 +00:00)


## additional stuff - can savely be ignored:

In [101]:
def p_at(qid, p=10, queries=queries, gold=gold, docs=ourdocs):
    """
    takes a query id and returns the precision at 10 for that query
    gold contains the 100 best documents for each query
    """
    query = queries[queries.qid==qid]['query']
    # print(query)
    qv = vectorizer.transform(query) # query vector
    xqv = X*qv.T # cosine similarity between query and all documents
    pred10i = np.argpartition(xqv.A.flat, -p)[-p:] # top 10
    # display(docs.loc[pred10i],gold[gold.qid==qid].docid,docs.loc[pred10i].isin(gold[gold.qid==qid].docid).docid)
    qgold = gold[(gold.qid==qid) & (gold['rank']<=p)].docid
    intersection = np.intersect1d(docs.loc[pred10i].docid, qgold) # intersection of top p and gold
    return len(intersection)/p

print(p_at(251898))
print(p_at(687888))

0.4
0.8
time: 834 ms (started: 2024-04-05 00:26:08 +00:00)


In [102]:
training_at_10 = training_queries.qid.progress_apply(p_at)

  0%|          | 0/500 [00:00<?, ?it/s]

time: 3min 11s (started: 2024-04-05 00:26:12 +00:00)


In [103]:
# how many of the 500 test queries have at least one relevant document in the top 10?
len(training_at_10[training_at_10>0])

430

time: 3.73 ms (started: 2024-04-05 00:29:23 +00:00)


In [104]:
# average number of relevant documents in the top 10
training_at_10[training_at_10>0].mean()

0.2744186046511628

time: 3.41 ms (started: 2024-04-05 00:29:23 +00:00)


In [105]:
training_at_10[training_at_10>0]

0      0.8
1      0.1
2      0.1
3      0.3
4      0.3
      ... 
495    0.1
496    0.2
497    0.5
498    0.2
499    0.6
Name: qid, Length: 430, dtype: float64

time: 4.63 ms (started: 2024-04-05 00:29:23 +00:00)


# reranking

In [43]:
# %pip install sentence_transformers -U

time: 208 µs (started: 2024-04-04 23:16:17 +00:00)


In [106]:
from sentence_transformers import CrossEncoder

# Load the model, here we use our base sized model
model = CrossEncoder("mixedbread-ai/mxbai-rerank-xsmall-v1")



time: 693 ms (started: 2024-04-05 00:29:23 +00:00)


In [107]:

# Example query and documents
query = "Who wrote 'To Kill a Mockingbird'?"
documents = [
    "'To Kill a Mockingbird' is a novel by Harper Lee published in 1960. It was immediately successful, winning the Pulitzer Prize, and has become a classic of modern American literature.",
    "The novel 'Moby-Dick' was written by Herman Melville and first published in 1851. It is considered a masterpiece of American literature and deals with complex themes of obsession, revenge, and the conflict between good and evil.",
    "Harper Lee, an American novelist widely known for her novel 'To Kill a Mockingbird', was born in 1926 in Monroeville, Alabama. She received the Pulitzer Prize for Fiction in 1961.",
    "Jane Austen was an English novelist known primarily for her six major novels, which interpret, critique and comment upon the British landed gentry at the end of the 18th century.",
    "The 'Harry Potter' series, which consists of seven fantasy novels written by British author J.K. Rowling, is among the most popular and critically acclaimed books of the modern era.",
    "'The Great Gatsby', a novel written by American author F. Scott Fitzgerald, was published in 1925. The story is set in the Jazz Age and follows the life of millionaire Jay Gatsby and his pursuit of Daisy Buchanan."
]

time: 512 µs (started: 2024-04-05 00:29:24 +00:00)


In [108]:
results = model.rank(query, documents, return_documents=True, top_k=3)

time: 126 ms (started: 2024-04-05 00:29:24 +00:00)


In [109]:
results

[{'corpus_id': 0,
  'score': 0.99463475,
  'text': "'To Kill a Mockingbird' is a novel by Harper Lee published in 1960. It was immediately successful, winning the Pulitzer Prize, and has become a classic of modern American literature."},
 {'corpus_id': 2,
  'score': 0.98541903,
  'text': "Harper Lee, an American novelist widely known for her novel 'To Kill a Mockingbird', was born in 1926 in Monroeville, Alabama. She received the Pulitzer Prize for Fiction in 1961."},
 {'corpus_id': 3,
  'score': 0.58034474,
  'text': 'Jane Austen was an English novelist known primarily for her six major novels, which interpret, critique and comment upon the British landed gentry at the end of the 18th century.'}]

time: 3.12 ms (started: 2024-04-05 00:29:24 +00:00)


time: 9.21 ms (started: 2024-04-04 23:16:21 +00:00)


time: 15.8 ms (started: 2024-04-04 23:16:21 +00:00)
